# Data

In [108]:
import torch 
from torch import nn , optim
from tqdm import tqdm
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
Pieces = 200
#                 #CH1 CH2
# X = torch.tensor([[1,2], #Music Sample 1
#                      [3,4], #Music Sample 2
#                      [5,6], #Music Sample 2
#                      [7,8]],dtype=torch.float32)/500#Music Sample 5
jazz_train = torch.rand(Pieces,10,2,dtype=torch.float32)# N C L
rock_train = torch.rand(Pieces,10,2,dtype=torch.float32)# N C L
jazz_train[:, :, 0] += 30
jazz_train[:, :, 1] += 60
rock_train[:, :, 0] += 80
rock_train[:, :, 1] += 10
y_genre_train = torch.cat(                                           
    (torch.zeros(Pieces, dtype=torch.long),                          # 0 = jazz
     torch.ones(Pieces, dtype=torch.long)),                          # 1 = rock
    dim=0
)    

X_NLH = torch.cat((jazz_train,rock_train),0)/100
N,L,Hin = X_NLH.shape

y = y_genre_train

batchsize = 8
# hn =  # latent data/ hidden data

X_train,X_test,y_train,y_test = train_test_split(X_NLH,y)# defult split  -> 75% trainning 25% testing
trainning_dataset = TensorDataset(X_train,y_train)
tranning_dataLoader = DataLoader(trainning_dataset,batch_size=batchsize)

test_dataset = TensorDataset(X_test,y_test)
test_dataLoader = DataLoader(test_dataset,batch_size=batchsize)
for count,(xb,yb) in enumerate(tranning_dataLoader): # enumerate = python function loop counter
    print(count,yb)

cpu
0 tensor([0, 1, 0, 0, 1, 0, 0, 0])
1 tensor([0, 0, 0, 0, 0, 0, 0, 0])
2 tensor([0, 0, 1, 0, 0, 1, 0, 1])
3 tensor([1, 0, 0, 1, 1, 0, 0, 1])
4 tensor([1, 0, 1, 0, 0, 1, 1, 1])
5 tensor([0, 1, 1, 1, 0, 1, 1, 1])
6 tensor([1, 0, 1, 0, 1, 0, 0, 0])
7 tensor([1, 0, 1, 0, 0, 1, 0, 1])
8 tensor([1, 1, 0, 1, 0, 1, 0, 0])
9 tensor([0, 0, 0, 0, 1, 0, 1, 0])
10 tensor([0, 0, 0, 1, 0, 1, 1, 1])
11 tensor([0, 1, 1, 0, 0, 1, 1, 0])
12 tensor([1, 0, 1, 0, 1, 1, 1, 0])
13 tensor([0, 0, 1, 1, 1, 0, 0, 1])
14 tensor([0, 1, 1, 1, 0, 1, 1, 1])
15 tensor([1, 0, 1, 1, 1, 0, 0, 1])
16 tensor([0, 1, 1, 1, 1, 1, 0, 1])
17 tensor([0, 0, 0, 1, 1, 0, 0, 1])
18 tensor([1, 1, 1, 0, 1, 0, 1, 0])
19 tensor([0, 0, 0, 1, 1, 1, 1, 1])
20 tensor([1, 0, 1, 0, 1, 0, 1, 1])
21 tensor([1, 1, 1, 1, 1, 1, 0, 0])
22 tensor([1, 0, 0, 1, 1, 1, 0, 0])
23 tensor([1, 1, 0, 0, 0, 1, 0, 1])
24 tensor([0, 0, 1, 0, 1, 1, 1, 1])
25 tensor([0, 0, 1, 1, 1, 1, 0, 1])
26 tensor([0, 0, 0, 1, 0, 0, 0, 1])
27 tensor([0, 1, 1, 1, 0, 1, 1, 0]

# Hyperparametmers

In [109]:
Hout = 5 # Size of the hidden state
n_classes = 2 # Number of classes

ht_1 = torch.zeros(1,batchsize, Hout)


Wx = torch.rand(Hout,Hin) # Shape of (hidden_size(Hout),input size(Hin))
bx = torch.rand(Hout) # Shape of (hidden_size)

Wh = torch.rand(Hout,Hout)#shape of (hidden_size, hidden_size)
bh = torch.rand(Hout)#Shape of (hidden_size)




# Backbone

In [110]:
backbone = nn.RNN(input_size=Hin,hidden_size=Hout,batch_first=True)
backbone.weight_ih_l0.data = Wx
backbone.bias_ih_l0.data = bx
backbone.weight_hh_l0.data = Wh
backbone.bias_hh_l0.data = bh
# output, hn = backbone(X, h0)
# print(f"output {output}\n, hn {hn}")


# Head

In [111]:

head = nn.Linear(in_features=Hout,out_features=n_classes)# two classes

# Model

In [112]:
model = nn.ModuleList([backbone,head]).to(device)

# Trainning

In [113]:

loss_fn = nn.CrossEntropyLoss()# compute the error 
optim_fn = optim.Adam(model.parameters(),lr=0.01) # Stochastic Gradient descent
for epoch in tqdm(range(1)):
    for xb,yb in tranning_dataLoader: 
        xb,yb= xb.to(device),yb.to(device)
        output, ht = backbone(xb) # find the output of the model
        logits = torch.squeeze(head(ht),dim=0) # Logits is the finale Output of the nn.Linear / # ht -> head -> outcome logits
        loss = loss_fn(logits, yb)  #Out is the ML think is the answer, yb the real answer 
        optim_fn.zero_grad() # get ready for the slope
        loss.backward()# find where is it down the slope
        optim_fn.step()# Take a step down
        # print(loss)
        # head(ht) 
        # Xb -> Backbone RNN -> ht -> Head Linear -> y_pred
      



100%|██████████| 1/1 [00:00<00:00, 33.23it/s]


In [ ]:
acc = 0
for xb,yb in test_dataLoader:         
        xb,yb= xb.to(device),yb.to(device)
        output, ht = backbone(xb) # find the output of the model
        logits = torch.squeeze(head(ht),dim=0) # Logits is the finale Output of the nn.Linear / # ht -> head -> outcome logits
        ypred = logits.argmax(axis = 1)
        result = (ypred == yb).sum()/len(yb)*100
        if result == 0:
                acc += 1
        
        # print(ypred,yb,logits)
        # TODO finish the percentage using
        # Loop with in batches
        
print(acc)
        

RuntimeError: Boolean value of Tensor with more than one value is ambiguous

In [115]:
a = torch.tensor([0,1,1])
b = torch.tensor([0,1,0])

print(((a == b).sum()/len(b))*100)

tensor(66.6667)


In [116]:
# ht_1 = h0 # ht - 1
# for t in range(L):
#     x = X[:,t,:] # Shape of (N,Hin)
#     wxTx = torch.matmul(x,Wx.T)
#     whTh = torch.matmul(ht_1, Wh.T)
#     ht = torch.tanh(wxTx+bx+whTh+bh)
#     ht_1 = ht
#     print(ht)
    
# torch.allclose(hn,ht)
# # print(output)
# # Rnn(X,1)


HW.

Finish the RNN Cell
N = multiable number
